In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/12-class-ssvep-eeg-data/s2.mat
/kaggle/input/12-class-ssvep-eeg-data/s5.mat
/kaggle/input/12-class-ssvep-eeg-data/s6.mat
/kaggle/input/12-class-ssvep-eeg-data/s10.mat
/kaggle/input/12-class-ssvep-eeg-data/s8.mat
/kaggle/input/12-class-ssvep-eeg-data/s7.mat
/kaggle/input/12-class-ssvep-eeg-data/s1.mat
/kaggle/input/12-class-ssvep-eeg-data/s3.mat
/kaggle/input/12-class-ssvep-eeg-data/s4.mat
/kaggle/input/12-class-ssvep-eeg-data/s9.mat


# 12 Class SSVEP Dataset using CCA

## 1. Import libs and utils

In [2]:
import os
import sys 
import scipy.io as sio
# ssvep utils 
import ssvep_ultils as su 
from sklearn.metrics import confusion_matrix
from sklearn.cross_decomposition import CCA

/kaggle/input/12-class-ssvep-eeg-data/s2.mat
/kaggle/input/12-class-ssvep-eeg-data/s5.mat
/kaggle/input/12-class-ssvep-eeg-data/s6.mat
/kaggle/input/12-class-ssvep-eeg-data/s10.mat
/kaggle/input/12-class-ssvep-eeg-data/s8.mat
/kaggle/input/12-class-ssvep-eeg-data/s7.mat
/kaggle/input/12-class-ssvep-eeg-data/s1.mat
/kaggle/input/12-class-ssvep-eeg-data/s3.mat
/kaggle/input/12-class-ssvep-eeg-data/s4.mat
/kaggle/input/12-class-ssvep-eeg-data/s9.mat


## 2. Import data

In [3]:
import os
import numpy as np
from scipy.io import loadmat

data_files = [
    '/kaggle/input/12-class-ssvep-eeg-data/s1.mat',
    '/kaggle/input/12-class-ssvep-eeg-data/s2.mat',
    '/kaggle/input/12-class-ssvep-eeg-data/s3.mat',
    '/kaggle/input/12-class-ssvep-eeg-data/s4.mat',
    
    '/kaggle/input/12-class-ssvep-eeg-data/s5.mat',
    '/kaggle/input/12-class-ssvep-eeg-data/s6.mat',
    '/kaggle/input/12-class-ssvep-eeg-data/s7.mat',
    '/kaggle/input/12-class-ssvep-eeg-data/s8.mat',
    '/kaggle/input/12-class-ssvep-eeg-data/s9.mat',
    '/kaggle/input/12-class-ssvep-eeg-data/s10.mat',
]

all_segment_data = dict()
all_acc = list()
window_len = 1
shift_len = 1
sample_rate = 256
duration = int(window_len * sample_rate)
flicker_freq = np.array([9.25, 11.25, 13.25, 9.75, 11.75, 13.75, 
                         10.25, 12.25, 14.25, 10.75, 12.75, 14.75])

for file_path in data_files:
    subject_id = os.path.basename(file_path).split('.')[0] 
    print(f"Loading data for {subject_id} from {file_path}...")
    data = loadmat(file_path) 
    all_segment_data[subject_id] = data

print("Data loading completed.")

# check
for subject, data in all_segment_data.items():
    print(f"Subject {subject}: Keys in data - {list(data.keys())}")


Loading data for s1 from /kaggle/input/12-class-ssvep-eeg-data/s1.mat...
Loading data for s2 from /kaggle/input/12-class-ssvep-eeg-data/s2.mat...
Loading data for s3 from /kaggle/input/12-class-ssvep-eeg-data/s3.mat...
Loading data for s4 from /kaggle/input/12-class-ssvep-eeg-data/s4.mat...
Loading data for s5 from /kaggle/input/12-class-ssvep-eeg-data/s5.mat...
Loading data for s6 from /kaggle/input/12-class-ssvep-eeg-data/s6.mat...
Loading data for s7 from /kaggle/input/12-class-ssvep-eeg-data/s7.mat...
Loading data for s8 from /kaggle/input/12-class-ssvep-eeg-data/s8.mat...
Loading data for s9 from /kaggle/input/12-class-ssvep-eeg-data/s9.mat...
Loading data for s10 from /kaggle/input/12-class-ssvep-eeg-data/s10.mat...
Data loading completed.
Subject s1: Keys in data - ['__header__', '__version__', '__globals__', 'ans', 'train', 'list_sub', 'sub_i', 'eeg']
Subject s2: Keys in data - ['__header__', '__version__', '__globals__', 'ans', 'train', 'list_sub', 'sub_i', 'eeg']
Subject s3: 

### Get CCA reference signals 

In [4]:
def get_cca_reference_signals(data_len, target_freq, sampling_rate):
    reference_signals = []
    t = np.arange(0, (data_len/(sampling_rate)), step=1.0/(sampling_rate))
    reference_signals.append(np.sin(np.pi*2*target_freq*t))
    reference_signals.append(np.cos(np.pi*2*target_freq*t))
    reference_signals.append(np.sin(np.pi*4*target_freq*t))
    reference_signals.append(np.cos(np.pi*4*target_freq*t))
    reference_signals = np.array(reference_signals)
    
    return reference_signals

### Explain this

In [5]:
def find_correlation(n_components, np_buffer, freq):
    cca = CCA(n_components)
    corr = np.zeros(n_components)
    result = np.zeros(freq.shape[0])
    for freq_idx in range(0,freq.shape[0]):
        cca.fit(np_buffer.T,np.squeeze(freq[freq_idx, :, :]).T)
        O1_a,O1_b = cca.transform(np_buffer.T, np.squeeze(freq[freq_idx, :, :]).T)
        ind_val = 0
        for ind_val in range(0,n_components):
            corr[ind_val] = np.corrcoef(O1_a[: ,ind_val], O1_b[:, ind_val])[0 ,1]
            result[freq_idx] = np.max(corr)
    
    return result

### Explain this

In [6]:
def cca_classify(segmented_data, reference_templates):
    predicted_class = []
    labels = []
    for target in range(0, segmented_data.shape[0]):
        for trial in range(0, segmented_data.shape[2]):
            for segment in range(0, segmented_data.shape[3]):
                labels.append(target)
                result = find_correlation(1, segmented_data[target, :, trial, segment, :], 
                                      reference_templates)
                predicted_class.append(np.argmax(result)+1)
    labels = np.array(labels)+1
    predicted_class = np.array(predicted_class)

    return labels, predicted_class

### Filtering data and segment epochs

In [7]:
for idx, file_path in enumerate(data_files):
    dataset = sio.loadmat(file_path)
    eeg = np.array(dataset['eeg'], dtype='float32')
    
    num_classes = eeg.shape[0]
    n_ch = eeg.shape[1]
    total_trial_len = eeg.shape[2]
    num_trials = eeg.shape[3]
    
    filtered_data = su.get_filtered_eeg(eeg, 6, 80, 4, sample_rate)
    all_segment_data[f's{idx+1}'] = su.get_segmented_epochs(
        filtered_data, window_len, shift_len, sample_rate
    )


### Generating the required sinusoidal templates for the given 12-class SSVEP classification

In [8]:
reference_templates = []
for fr in range(0, len(flicker_freq)):
    reference_templates.append(get_cca_reference_signals(duration, flicker_freq[fr], sample_rate))
reference_templates = np.array(reference_templates, dtype='float32')

## 3. Perform CCA on the Segmented epochs 

In [9]:
for subject in all_segment_data.keys():
    labels, predicted_class = cca_classify(all_segment_data[subject], reference_templates)
    c_mat = confusion_matrix(labels, predicted_class)
    accuracy = np.divide(np.trace(c_mat), np.sum(np.sum(c_mat)))
    all_acc.append(accuracy)
    print(f'Subject: {subject}, Accuracy: {accuracy*100} %')

Subject: s1, Accuracy: 29.166666666666668 %
Subject: s2, Accuracy: 26.25 %
Subject: s3, Accuracy: 59.44444444444444 %
Subject: s4, Accuracy: 80.27777777777779 %
Subject: s5, Accuracy: 52.361111111111114 %
Subject: s6, Accuracy: 87.22222222222223 %
Subject: s7, Accuracy: 69.16666666666667 %
Subject: s8, Accuracy: 96.66666666666667 %
Subject: s9, Accuracy: 66.38888888888889 %
Subject: s10, Accuracy: 65.27777777777779 %


### Overall Accuracy Across Subjects: 

In [10]:
all_acc = np.array(all_acc)
print(f'Overall Accuracy Across Subjects: {np.mean(all_acc)*100} %, std: {np.std(all_acc)*100} %')

Overall Accuracy Across Subjects: 63.22222222222222 %, std: 21.665580457103147 %
